In [21]:
# !pip install earthengine-api

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from datetime import timedelta
import ee
import math
scale = 0.009
#Experimentally found smaller = pinpoint location

In [ ]:
ee.Authenticate()
ee.Initialize()

In [15]:
fires = pd.read_csv('Gridded_Small_Fires.csv')

dts = []
for i in fires['Datetime']:
    dts.append(datetime.strptime(i, '%Y-%m-%d'))

fires['Datetime'] = dts
print(fires.head())

   Unnamed: 0  Longitude  Latitude   Datetime  Fire
0           0   -116.700    33.862 2020-09-18     1
1           1   -123.000    39.855 2020-09-10     1
2           2   -121.473    36.068 2020-09-07     1
3           3   -121.608    36.151 2020-08-21     1
4           4   -119.435    37.286 2020-09-21     1


In [16]:
dataset = ee.ImageCollection("MODIS/MYD09GA_006_NDVI")

feature_array = ['NDVI']

#Use the 16-day NDVI Modis data as well - seems more legit


In [18]:
#Transforms client-side ee.Image.getRegion array to pandas.DataFrame.
#List of bands is the list of features that we are interested in
def ee_array_to_df(arr, list_of_bands):
    df = pd.DataFrame(arr)
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    # df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()
    # print(len(df))

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    #Check for NaN values
#     print(df)
#     for i in df:
#         if math.isnan(float(df[i][0])):
#             marker = 1
#             raise Exception("NaN Value detected")

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]
    
    return df

In [19]:
# a_date = datetime. date(2015, 10, 10)
# days = datetime. timedelta(5)
# new_date = a_date - days. Subtract 5 days from a_date.
# print(new_date)

# print(fires['datetime'][0])
# print(type(fires['datetime'][0]))
# #Year-Month-Date
# a = fires['datetime'][0].strftime('%Y-%m-%d')
# print(a)
# print(type(a))
#returns [i_date, f_date]
#freq - e.g, 7 days per observation

#NOTE: Its EXCLUSIVE last date
#Data interpolation
def getDate(dt, freq):
#     freq -= 1
    half = int(freq/2)
    sub = timedelta(half)
    add = timedelta(freq - half)
    i_d = dt - sub
    f_d = dt + add
    
    i_d = i_d.strftime('%Y-%m-%d')
    f_d = f_d.strftime('%Y-%m-%d')
    return [i_d, f_d]

test = fires['Datetime'][0]
print(type(test))
#^ is 2010-05-19
print(getDate(test, 5))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
['2020-09-16', '2020-09-21']


In [23]:
#i_date - string, inclusive
#f_date - string, exclusive
def request(i_date, f_date, lon, lat, feature_array):
    cur = dataset.filter(ee.Filter.date(i_date, f_date))
    pos = ee.Geometry.Point(lon, lat)
    res = cur.getRegion(pos, scale).getInfo()
    # if(len(res) > 2):
        # print("more than one satellite entry for date range")
    df = ee_array_to_df(res,feature_array)
    return df

i_date = '2018-08-01'
f_date = '2018-08-02'
# test = dataset.filter(ee.Filter.date(i_date, f_date))
u_lon = -119.16
u_lat = 37.5650

# r_lon = -120.0
# r_lat = 37.8

# !! ___ TEST FOR FREQUENCY HERE ____ !!
print(request(i_date, f_date, u_lon, u_lat, feature_array))
# print(request(i_date, f_date, r_lon, r_lat, feature_array))
freq = 1

0           time   datetime      NDVI
0  1533081600000 2018-08-01  0.425548


In [24]:
all_exceptions = set()
def query(lon, lat, dat, daterange):
#     print(str(lon) + " " + str(lat) + " " + str(dat) + " " + str(daterange))
    try:
        i_d, f_d = getDate(dat, daterange)
        cur = request(i_d, f_d, lon, lat, feature_array)

        #Deal with NaNs --> They can be date dependent or location dependent
        idx = -1
        for i in cur:
            #Here, we can either do average, or take the closest date to the current date
            
            #Current approach: Take the closest date
            
            j = 0
            while j < len(cur):
                if not isinstance(cur[i][j], float):
                    j += 1
                    continue
                
                
                if not math.isnan(float(cur[i][j])):
                    idx = j
                    #Assumption: If one feature is non-NaN, then all values will be non-NaN

                    #THIS PART IS SCUFFED LMAO
                j+=1
                
        
        if idx == -1:
            raise Exception("All NaN Values -> try larger date range")
        
        #Essentially we're increasing range until we find the first date with all non-NaN values
        return cur.iloc[idx]
    except Exception as e:
        all_exceptions.add(e)
        print(e)
        print(str(lon) + " " + str(lat))
        return query(lon, lat, dat, daterange+1)

    
print(fires['Longitude'][4])
print(fires['Latitude'][4])
print(fires['datetime'][4])

temp = query(fires['Longitude'][4], fires['Latitude'][4], fires['datetime'][4], freq)
print(type(temp))
print(temp)


-119.435
37.286
2020-09-21 00:00:00
<class 'pandas.core.series.Series'>
0
time              1600646400000
datetime    2020-09-21 00:00:00
NDVI                   0.300939
Name: 0, dtype: object


In [ ]:
from random import randrange
from datetime import date
startdt = date(2015, 1, 1)
enddt = date(2021, 1, 1)
#2015-2020

all_set = set()
all_tuples = set()

for i in range(len(fires)):
    all_set.add((fires['Longitude'][i], fires['Latitude'][i]))
print(len(all))

all = []
for i in all_set:
    all.append(i)

nofires = []
nofiresz = 15000

idx = 0

In [ ]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

while idx < nofiresz:
    i = randrange(0, len(all))
    lon = all[i][0]
    lat = all[i][1]
    randomdt = random_date(startdt, enddt)

    tup = (lon, lat, randomdt)

    if(tup in all_tuples):
        continue

    all_tuples.add(tup)
    #two months buffer
    if(query(lon, lat, randomdt, 62)):
        nofires.append(tup)
        idx += 1
        print(tup)



In [ ]:
print(nofires)

In [ ]:
lats = []
lons = []
times = []

for i in nofires:
    lats.append(i[1])
    lons.append(i[0])
    times.append(i[2])

res = pd.DataFrame()
res['Longitude'] = lons
res['Latitude'] = lats
res['Datetime'] = times

res.to_csv('15kNoFires.csv')


In [ ]:
#combine everything -- we only keep time, lon, lat, address, yes/no
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

res = pd.read_csv('2kCalFires.csv')
# print(res.head())

res['LocationString'] = res['Latitude'].map(str) + ',' + res['Longitude'].map(str)
print(res['LocationString'][1211])

for i in range(len(res)):
    l = float(res['Latitude'][i])
    
    if not (-90 <= l and l <= 90):
        res.at[i, 'Latitude'] = res['Longitude'][i]
        res.at[i, 'Longitude'] = l
print(res['LocationString'][1211])

locator = geopy.Nominatim(user_agent='need zipcode plz', timeout=10)
res['LocationString'] = res['Latitude'].map(str) + ',' + res['Longitude'].map(str)
# print(res['Longitude'])
print(len(res))

In [ ]:
idx = 0
final_addresses = []
notpossible = []
drops = []

In [ ]:
sz = len(res)

while idx < sz:
    try:
        location = locator.reverse(res['LocationString'][idx])
        
        #There are some mess ups -- 228 for example, they entered the long and lat backwards -- swaps to be made
        address = location.address

        if not " California," in address:
            print("NOT POSSIBLE AT: " + str(idx))
            raise Exception("Not possible")
            notpossible.append(idx)
        
        final_addresses.append(address)
        print(address)
        # if i % 1000 == 0:
        print("done with " + str(idx))
    except:
        drops.append(idx)
        print("error with " + str(idx))
    idx += 1

#THIS RES IS ACTUALLY THE CAL FIRES ONE NOW
#Drop, and then assign to address

# res['Address'] = final_addresses
# res.to_csv('final15kNoFires.csv')

In [ ]:
# print(len(notpossible))
# print(len(drops))
# print(drops)
# print(len(res))
res = res.drop(drops)
res = res.reset_index(drop=True)
print("now df is only " + str(len(res)))

In [ ]:
print(len(drops))
print(len(res))
print(len(notpossible))
res['Address'] = final_addresses
print(res.head())

In [ ]:
res.to_csv('finished2kFires.csv')